<a href="https://colab.research.google.com/github/jahid-github/Oulu-AI_Programing_Exercise/blob/main/JT_H1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import copy
import time
import fsspec
from IPython.display import clear_output

In [ ]:
link2data = 'https://github.com/MattiGiamatti/intro2ai/tree/2c77b0df567620f99df89a0731d50cabba1b7fd2/JT_H1_EN/imgs'

dir = './imgs'

if not os.path.exists(dir):
    print('Downloading data into {}'.format(dir))
    fs = fsspec.filesystem("github", org="MattiGiamatti", repo="intro2ai", ref="2c77b0df567620f99df89a0731d50cabba1b7fd2")
    fs.get('JT_H1_EN/imgs', dir, recursive=True)

# <center>521160P Introduction to artificial intelligence<br><br>Exercise #1<br><br>Search algorithms<br></center>

This exercise deals with depth-first search and breadth-search to solve the river crossing problem and creates heuristic evaluation functions for the 8-puzzle game. **Look return deadlines from moodle**. It is possible the get 4 points from this exercise (2p + 2p)

If you have any questions related to exercises or you face any problems during this exercise please use **moodle forum for programming exercise 1.**

**First fill in your group information (name and student number)**

# Group member information :

* **Member 1 :** `Md Jahidul Islam 2503404 `


# Task 1. Wolf, sheep, cabbage and shepherd river crossing puzzle with breadth-first and depth first algorithms

<br>
<div style="width:image width px; font-size:80%; text-align:center;">
    <center>
    <img src='imgs/rivercrossingpuzzle.png' width='700' height='auto' alt='alternate text' style='padding-bottom:0.5em;' />
    </center>
    <span>Figure 1. Wolf, sheep, cabbage and shepherd river crossing puzzle's initial state.</span>
</div>
<br>

Description: Shepherd is crossing a river with a rowing boat. He must get wolf, sheep and cabbage to other side of the river. The boat is small so it can hold only one item at the time. Shepherd's mission is restricted by following rules:

* Sheep cannot be left alone with the cabbage on the same side, for then the sheep eats the cabbage.

* Wolf cannot be left alone with the sheep on the same side, for then the wolf eats the sheep.

Help shepherd to find right order of moves to get to other side of the river with wolf, sheep and cabbage using **breadth-first** and **depth-first** algorithms. Let's define initial state according to figure 1, where all the objects are on the same side and on the island there is no one.

In [9]:
objects = sorted(['W','S','C','Shp']) # sorted() asures that objects are in alphabetical order
island = sorted([])
mainland = copy.deepcopy(objects)

print(f"Initially on mainland there is {mainland} and on island there is {island}")

For defining the restrictions of the problem we have implemented a few helper functions. These functions can be found in the JT_H1_EN_lib.py file. These functions are fully implemented.

In [ ]:
# Let's fetch the functions from JT_H1_EN_lib.py
from JT_H1_lib import generate_new_permited_mainlands, print_moves

## Uninformed search

### Depth-first

Implement **depth-first search** using pseudocode bellow. A few steps of the pseudocode is already implmented your job is to fill missing parts.
1. Initialize two empty lists open list and closed list. Open list corresponds to unexplored states and closed list corresponds to already treated states.
2. Define initial state which consists of two other lists mainland and path
3. Add initial state to open list
4. Loo through open list until empty
    1. Pick last element of the open list
    2. if mainland is not in closed_list
        1. Add mainland to closed list
        2. Check if mainland is same as goal
            1. if it is, return path
        3. Generate new mainlands using helper function *generate_new_permited_mainlands(objects, mainland)*
        4. loop through new mainlands one at a time Käy läpi silmukassa uudet mantereet yksi kerrallaan
            1. Solve new path by adding new mainland to current path
            2. add new state to open list which consists of new mainland and new path
5. print error message when open list is empty and no solution was not found.
        

In [3]:
def depth_first(objects, mainland):
    stack = [(mainland, [mainland])]
    visited = []

    while stack:
        current, path = stack.pop()
        if current in visited:
            continue
        visited.append(current)

        if len(current) == 0:
            return path

        for new_state in generate_new_permited_mainlands(objects, current):
            if new_state not in visited:
                stack.append((new_state, path + [new_state]))

    return None

### Breadth-first

In addition implemnt **breadth-first** algorithm. Code is similar to depth-first algorithm only difference is that in pseudocode above on part 4A instead of picking last element of list you pick the first one. Part of the pseudocode is already implemented fill the missing parts

In [2]:
def breadth_first(objects, mainland):
    queue = [(mainland, [mainland])]
    visited = []

    while queue:
        current, path = queue.pop(0)
        if current in visited:
            continue
        visited.append(current)

        if len(current) == 0:
            return path

        for new_state in generate_new_permited_mainlands(objects, current):
            if new_state not in visited:
                queue.append((new_state, path + [new_state]))

    return None

In [ ]:
mainland = objects.copy()

dfs_path = depth_first(objects, mainland)
bfs_path = breadth_first(objects, mainland)

print("DFS Moves:\n")
print_moves(objects, dfs_path)

print("\nBFS Moves:\n")
print_moves(objects, bfs_path)

print(f"\nDFS Steps: {len(dfs_path)-1}")
print(f"BFS Steps: {len(bfs_path)-1}")

**How many steps are required for depth-first and breadth-first algorithms to complete? How they differ from each other?**

`How many steps are required:`

* Depth-First Search (DFS) required 11 steps to complete the puzzle.

* Breadth-First Search (BFS) required 7 steps to complete the puzzle.

### DFS vs BFS Comparison Table

| Feature           | Depth-First Search (DFS)           | Breadth-First Search (BFS)           |
|------------------|------------------------------------|--------------------------------------|
| **Search Strategy**  | Explores one path deeply first     | Explores all paths level by level    |
| **Optimality**       | ❌ Not guaranteed to find shortest | ✅ Always finds the shortest path     |
| **Performance**      | May take longer or go in circles   | More efficient for shallow solutions |
| **Memory Usage**     | Lower memory usage                 | Higher memory usage                  |


# Task 2. 8-puzzle game with heuristic estimation functions

<br>
<div style="width:image width px; font-size:80%; text-align:center;">
    <center>
    <img src='imgs/eightpuzzle1.png' width='400' height='auto' alt='alternate text' style='padding-bottom:0.5em;' />
    </center>
    <span>Kuva 2. one initial state and goal state for 8-puzzle game.</span>
</div>
<br>

This task creates heuristic evaluation functions for $h(n)$ 8-puzzle game. 8-puzzle consists of eight sliding tile that are numbered from one to eight. Numbered tiles are placed on a 3x3 grid.One square from the grid is empty where you can either horizontally or vertically move one of the tiles next to it. When one of the tiles are moved to an empty tile, a previous place for the tile is left blank.  On 8-puzzle game, the purpose is to reach the target state as few moves from a known initial state. Figure 2 is an 8-puzzle game, an initial state and a target state.

8-puzzle game's search space is so large that with uninformed search algorithms it would take long. Because of that to find solution for game we use informed search algorithm **A\*-search**  

First let's implement a few essential helper functions for game.

In [ ]:
# Let's fetch the functions from JT_H1_EN_lib.py
from JT_H1_lib import print_state, generate_new_permited_states, sort_open_list, string

# Additionally
def calculate_heuristic_value(state, goal, heuristic_function):
    """
    Calculates value for estimation function depending on chosen heuristic function
    """
    if heuristic_function == "hamming":
        return calculate_hamming_distance(state, goal)
    elif heuristic_function == "city-block":
        return calculate_city_block_distance(state, goal)
    elif heuristic_function == "euclidean":
        return calculate_euclidean_distance(state, goal)
    else:
        print("Options are 'hamming', 'city-block' or 'euclidean'")

def a_star(initial_state, goal, heuristic_function):
    open_list = []
    closed_list = []
    path = []
    g_scores = {}
    f_scores = {}
    g_scores[string(initial_state)] = 0
    f_scores[string(initial_state)] = g_scores[string(initial_state)] + calculate_heuristic_value(initial_state, goal, heuristic_function)
    state = [initial_state, path]
    open_list.append(state)

    while len(open_list) != 0:
        open_list = sort_open_list(open_list, f_scores)
        state, path = open_list.pop(0)
        if state == goal:
            new_path = path + [state]
            return new_path, closed_list
        closed_list.append(state)
        new_states = generate_new_permited_states(state)
        for new_state in new_states:
            initial_g_score_value = g_scores[string(state)] + 1
            if new_state in closed_list and initial_g_score_value >= g_scores[string(new_state)]:
                continue
            if new_state not in closed_list or initial_g_score_value < g_scores[string(new_state)]:
                new_path = path + [state]
                g_scores[string(new_state)] = initial_g_score_value
                f_scores[string(new_state)] = g_scores[string(new_state)] + calculate_heuristic_value(new_state, goal, heuristic_function)
                if new_state not in open_list:
                    open_list.append([new_state, new_path])

    print("Solution couldn't be found")

def solve_game(initial_state, goal, heuristic_function):
    path, closed_list = a_star(initial_state, goal, heuristic_function)
    for state in path:
        clear_output(wait=True)
        print("Initial state:\n"+ print_state(initial_state))
        print(f"\nSolution with A*-search algorithm using {heuristic_function} distance:\n\n" + print_state(state))
        time.sleep(1)
    print(f"Solution was found in {len(path)-1} moves. And to find those algorithm searched in total {len(closed_list)} states")

## Heuristic estimation functions

The A * search algorithm requires a heuristic estimation function h(n) i.e. a cost estimate, from the state under study to the target state. A typical solution for a game is about 20 moves long and its branching factor i.e. how many moves can be performed on average in one turn is 2.66. This means that a thorough search from the depth of 20 movements goes through $2.66 ^ {20} = 3.15 * 10 ^ {8}$ space. For this reason, it is especially important to find the best possible heuristic function for an 8-puzzle game. Good options for heuristic functions are **Hamming distance**, **city-block distance** (also known as manhattan distance) and **Euclidean distance**.

* Hamming Distance: Counts the number of tiles in the wrong places. A state with few tiles in the wrong places is presumably closer to the target state than, for example, a state with all tiles in the wrong places.

* City-block distance: Calculates the horizontal and vertical absolute distance for each tile from the corresponding target space tile and finally sums them together. $h_{2}(n) = \sum\limits_{i = 1} ^ {8} | a_ {ix} - b_ {ix} | + | a_ {iy} - b_ {iy} |$, where $ a_ {ix} $ is the x-coordinate of the i-th tile of the initial state and $ b_ {iy} $ is the y-coordinate of the i-th tile of the target state.

* Euclidean distance: Calculates the sum of the horizontal and vertical distances squared for each slab from the corresponding target space slab, takes the square root of the sum, and finally sums the square roots together. $h_{3}(n) = \sum\limits_{i=1}^{8} \sqrt{(a_{ix} - b_{ix})^2 + (a_{iy} - b_{iy})^2}$.

The Hamming distance for the situation in Figure 2 is $h_ {1}(n) = 4$, because four of the initial state tiles are in the wrong places compared to the target state. The city-block distance is calculated for Figure 2 as follows: $ h_ {2} (n) = 1 + 0 + 0 + 3 + 2 + 0 + 2 + 0 = 8$. The Euclidean distance, in turn, for the situation in Figure 2 is: $h_ {3}(n) = 1 + 0 + 0 + \sqrt {5} + \sqrt {2} + 0 + 2 + 0 \approx 6.65$. The blank tile in the initial state has no effect on the values of the heuristic functions.

In the A * search algorithm, the depth of the game tree has been used as the path cost estimate $g(n)$. In the algorithm, unexamined states are stored in an open list and processed states in a closed list. Finally, after the algorithm finds a solution, a path is built from the initial state to the target state.

### Euclidean distance heuristic function

Heuristic evaluation functions take the state under study and the target state as arguments. The state under study and the target state are implemented as lists of 9 items, where a value of 0 represents an empty tile. The values in the list run from the upper left corner of the game board to the lower right corner. The initial state in the list in Figure 2 is ***initial state = [7,2,3,1,0,6,5,8,4]*** and the target state is ***target state = [1,2,3,4,5,6,7,8,0]***.

As an example a heuristic Euclidean-distance estimation function is implemented. Your task is to implement the Hamming-distance estimation function and the city-block-distance estimation function and test their performance for the initial state and the target state in Figure 2.

In [4]:
import math

def calculate_euclidean_distance(state, goal):
    distance = 0
    for tile in range(1, 9):
        idx_state = state.index(tile)
        idx_goal = goal.index(tile)
        x1, y1 = idx_state % 3, idx_state // 3
        x2, y2 = idx_goal % 3, idx_goal // 3
        distance += math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    return distance


check that euclidean distance heuristic function returns ~6,65 which was calculated above

In [5]:
# Define initial state and goal according to figure 2
initial_state = [7,2,3,1,0,6,5,8,4]
goal = [1,2,3,4,5,6,7,8,0]

# Check that function returns correct value
print('Euclideam distance for initial state is: {}'.format(calculate_euclidean_distance(initial_state, goal)))

Euclideam distance for initial state is: 6.650281539872885


After this solve game for initial state using euclidean distance as heuristic function

In [9]:
from JT_H1_lib import generate_new_permited_states, print_state, string, sort_open_list
import copy

def a_star_solver(initial_state, goal_state, heuristic_func):
    open_list = [(initial_state, [initial_state])]
    closed_list = []
    f_scores = {string(initial_state): heuristic_func(initial_state, goal_state)}

    while open_list:
        state, path = open_list.pop(0)

        if state == goal_state:
            return path

        closed_list.append(state)

        for new_state in generate_new_permited_states(state):
            if new_state not in closed_list:
                new_path = copy.deepcopy(path)
                new_path.append(new_state)
                f_scores[string(new_state)] = len(new_path) + heuristic_func(new_state, goal_state)
                open_list.append((new_state, new_path))

        open_list = sort_open_list(open_list, f_scores)
    return None

In [9]:

# Solve game using euclidean distance
path = a_star_solver(initial_state, goal, calculate_euclidean_distance)

for i, state in enumerate(path):
    print(f"Step {i}:\n{print_state(state)}")

print(f"Total Steps: {len(path) - 1}")

### Hamming distance estimation function

Implement heuristic hamming distance function

In [ ]:
def calculate_hamming_distance(state, goal):
    return sum([1 for i in range(9) if state[i] != 0 and state[i] != goal[i]])

Confirm that calculate_hamming_distance function returns 4 which was calculated above

In [ ]:
# Confirm that right value is returned
print('Hamming-distance for initial state is: {}'.format(calculate_hamming_distance(initial_state, goal)))

After this solve game using hamming distance

In [ ]:
# Solve game using Hamming distance
path = a_star_solver(initial_state, goal, calculate_hamming_distance)

for i, state in enumerate(path):
    print(f"Step {i}:\n{print_state(state)}")

print(f"Total Steps: {len(path) - 1}")


### City-block distance estimation function

Implement heuristic city-block distance function

In [ ]:
def calculate_city_block_distance(state, goal):
    distance = 0
    for tile in range(1, 9):
        idx_state = state.index(tile)
        idx_goal = goal.index(tile)
        x1, y1 = idx_state % 3, idx_state // 3
        x2, y2 = idx_goal % 3, idx_goal // 3
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance


Confirm that city-block-distance is calculated correctly. According to calculation above it should be 8

In [ ]:
# Confirm that function returns correct value
print('City-block distance for initial state is: {}'.format(calculate_city_block_distance(initial_state, goal)))

After this let's solve game with city-block distance

In [ ]:
# Solve game using city-block (Manhattan) distance
path = a_star_solver(initial_state, goal, calculate_city_block_distance)

for i, state in enumerate(path):
    print(f"Step {i}:\n{print_state(state)}")

print(f"Total Steps: {len(path) - 1}")


**Which function was best for this problem based on ammount of explored states?**
### Which heuristic was best?

| Heuristic         | Total Steps | Notes                                      |
|------------------|-------------|--------------------------------------------|
| Euclidean         | 1           | Accurate but uses floating point math      |
| Hamming           | 1           | Simple but less informative                |
| City-Block (Manhattan) | 1     | Very effective and widely used in puzzles  |

**Conclusion:**
All three heuristics returned the optimal path (1 step) for this easy test case.  
However, for more complex puzzles:
- **City-Block** tends to perform best (simple and accurate).
- **Hamming** is too simplistic.
- **Euclidean** is precise but more computationally expensive.

-- For 8-puzzle, **City-Block** is usually the best balance.


# Feedback
Finally answer the following questions

**How much time did you spent on this exercise?**

`Almost 15 Hours`

**Did you face any problems during this exercise? Was notebook material enough for doing this exercise?**

** Faced different problem and took help from AI and Use colab

**Other feedback related to this exercise**

** First time i did not catch how to solve it and doing step by step

# Returning the exercise

1. Before you return this notebook **make sure that code in notebook works properly** and returns all necessary values by choosing from menu `Kernel -> Restart & Run All`! also make sure you have answer all questions on **bold**
2. Empty all outputs and variables by choosing from menu `Kernel -> Restart & Clear Output`. This step reduces the size of notebook remarkably
3. Rename this notebook in following format **`JT_H1_[student_number(s)].ipynb`** (e.g `JT_H1_1234567.ipynb` or if you have group `JT_H1_1234567_2345678_3456789.ipynb`)
4. Return **only** solved notebook(`file ending with .ipynb`) to moodle programming exercise 1. Everyone must return file to moodle even if you work in a group. **Don't include working directory or other files** when you return the exercise.
5. Finally go answer questions in moodle related to programming exercise 1. Everyone in your group must answer to questions.